# Optimizing with Tensorboard

The most popular use of Tensorboard is to manage multiple models. When training there are a lot of fine tuning to do that it gets tedious. Some parameters like the structure of the neural network, the optimizer used, the learning rate in the optimizer, etc. permutates to thousands to millions of possible models.



The things we are going to tweak in:

```model.add(Conv2D(64, (3,3), input_shape(X.shape[1:]))```

are the kernel size ```(3,3)```, number of layers, number of nodes per layer ```64```, and the choice of having a dense layer at the end.

Here we are permutating to all the possibilities based on the options above.

Now let's see it come together.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
pickle_in = open("pickles/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("pickles/y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [4]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

In [5]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
            
            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3, callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 8s 445us/step - loss: 0.6288 - acc: 0.6492 - val_loss: 0.5843 - val_acc: 0.7071
Epoch 2/10
17462/17462 [==============================] - 7s 376us/step - loss: 0.5360 - acc: 0.7347 - val_loss: 0.5454 - val_acc: 0.7321
Epoch 3/10
17462/17462 [==============================] - 7s 373us/step - loss: 0.4999 - acc: 0.7582 - val_loss: 0.5478 - val_acc: 0.7263
Epoch 4/10
17462/17462 [==============================] - 7s 375us/step - loss: 0.4671 - acc: 0.7849 - val_loss: 0.5315 - val_acc: 0.7417
Epoch 5/10
17462/17462 [==============================] - 7s 381us/step - loss: 0.4451 - acc: 0.7933 - val_loss: 0.5707 - val_acc: 0.7147
Epoch 6/10
17462/17462 [==============================] - 6s 369us/step - loss: 0.4259 - acc: 0.8041 - val_loss: 0.5573 - val_acc: 0.7346
Epoch 7/10
17462/17462 [==============================] - 6s 364us/step - loss: 0.4094 - acc: 0.8146 - val_loss: 0.5740 - v

17462/17462 [==============================] - 9s 517us/step - loss: 0.3085 - acc: 0.8654 - val_loss: 0.3900 - val_acc: 0.8351
Epoch 9/10
17462/17462 [==============================] - 9s 506us/step - loss: 0.2850 - acc: 0.8786 - val_loss: 0.3936 - val_acc: 0.8298
Epoch 10/10
17462/17462 [==============================] - 9s 516us/step - loss: 0.2654 - acc: 0.8860 - val_loss: 0.4133 - val_acc: 0.8196
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 11s 634us/step - loss: 0.6217 - acc: 0.6557 - val_loss: 0.5829 - val_acc: 0.6933
Epoch 2/10
17462/17462 [==============================] - 10s 590us/step - loss: 0.5321 - acc: 0.7409 - val_loss: 0.5540 - val_acc: 0.7229
Epoch 3/10
17462/17462 [==============================] - 10s 574us/step - loss: 0.4857 - acc: 0.7685 - val_loss: 0.5597 - val_acc: 0.7167
Epoch 4/10
17462/17462 [==============================] - 10s 584us/step - loss: 0.4525 - acc: 0.7886 - val_loss: 0.5619 - val_acc

17462/17462 [==============================] - 8s 433us/step - loss: 0.4192 - acc: 0.8061 - val_loss: 0.4780 - val_acc: 0.7743
Epoch 6/10
17462/17462 [==============================] - 8s 437us/step - loss: 0.3887 - acc: 0.8233 - val_loss: 0.4394 - val_acc: 0.7953
Epoch 7/10
17462/17462 [==============================] - 7s 429us/step - loss: 0.3607 - acc: 0.8360 - val_loss: 0.4120 - val_acc: 0.8156
Epoch 8/10
17462/17462 [==============================] - 7s 427us/step - loss: 0.3347 - acc: 0.8512 - val_loss: 0.4051 - val_acc: 0.8176
Epoch 9/10
17462/17462 [==============================] - 7s 429us/step - loss: 0.3106 - acc: 0.8629 - val_loss: 0.3990 - val_acc: 0.8268
Epoch 10/10
17462/17462 [==============================] - 8s 436us/step - loss: 0.2868 - acc: 0.8740 - val_loss: 0.4044 - val_acc: 0.8244
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 10s 561us/step - loss: 0.6077 - acc: 0.6717 - val_loss: 0.5616 - val_acc: 0

17462/17462 [==============================] - 14s 773us/step - loss: 0.5075 - acc: 0.7499 - val_loss: 0.4696 - val_acc: 0.7817
Epoch 3/10
17462/17462 [==============================] - 14s 781us/step - loss: 0.4330 - acc: 0.7966 - val_loss: 0.4272 - val_acc: 0.8037
Epoch 4/10
17462/17462 [==============================] - 14s 774us/step - loss: 0.3801 - acc: 0.8273 - val_loss: 0.3894 - val_acc: 0.8287
Epoch 5/10
17462/17462 [==============================] - 14s 781us/step - loss: 0.3332 - acc: 0.8507 - val_loss: 0.3810 - val_acc: 0.8272
Epoch 6/10
17462/17462 [==============================] - 14s 774us/step - loss: 0.2835 - acc: 0.8790 - val_loss: 0.3903 - val_acc: 0.8247
Epoch 7/10
17462/17462 [==============================] - 13s 772us/step - loss: 0.2403 - acc: 0.8970 - val_loss: 0.4046 - val_acc: 0.8300
Epoch 8/10
17462/17462 [==============================] - 14s 779us/step - loss: 0.1912 - acc: 0.9217 - val_loss: 0.4385 - val_acc: 0.8343
Epoch 9/10
17462/17462 [==============

17462/17462 [==============================] - 10s 549us/step - loss: 0.0769 - acc: 0.9717 - val_loss: 1.0606 - val_acc: 0.7555
Epoch 10/10
17462/17462 [==============================] - 10s 549us/step - loss: 0.0556 - acc: 0.9806 - val_loss: 1.1715 - val_acc: 0.7541
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 12s 675us/step - loss: 0.6452 - acc: 0.6179 - val_loss: 0.5918 - val_acc: 0.6889
Epoch 2/10
17462/17462 [==============================] - 10s 574us/step - loss: 0.5214 - acc: 0.7425 - val_loss: 0.4873 - val_acc: 0.7667
Epoch 3/10
17462/17462 [==============================] - 10s 565us/step - loss: 0.4370 - acc: 0.8000 - val_loss: 0.4352 - val_acc: 0.7978
Epoch 4/10
17462/17462 [==============================] - 11s 640us/step - loss: 0.3853 - acc: 0.8273 - val_loss: 0.3955 - val_acc: 0.8187
Epoch 5/10
17462/17462 [==============================] - 10s 564us/step - loss: 0.3472 - acc: 0.8457 - val_loss: 0.4212 - val_